In [1]:
import polars as pl
from raphtory import Graph

df = pl.read_csv("lotr.csv");

print(df);


shape: (2648, 3)
┌───────────┬─────────┬───────┐
│ Gandalf   ┆ Elrond  ┆ 33    │
│ ---       ┆ ---     ┆ ---   │
│ str       ┆ str     ┆ i64   │
╞═══════════╪═════════╪═══════╡
│ Frodo     ┆ Bilbo   ┆ 114   │
│ Blanco    ┆ Marcho  ┆ 146   │
│ Frodo     ┆ Bilbo   ┆ 205   │
│ Thorin    ┆ Gandalf ┆ 270   │
│ ...       ┆ ...     ┆ ...   │
│ Merry     ┆ Sam     ┆ 32666 │
│ Galadriel ┆ Sam     ┆ 32666 │
│ Pippin    ┆ Merry   ┆ 32671 │
│ Pippin    ┆ Merry   ┆ 32674 │
└───────────┴─────────┴───────┘
